## 4.9 环境和分布偏移
前面我们学习了许多机器学习的实际应用，将模型拟合各种数据集。然而，我们从来没有想过数据集最初从哪里来？以及我们计划最终如何处理模型的输出？通常情况下，开发人员会拥有一些数据且急于开发模型，而不关注这些基本问题。

许多失败的机器学习部署(即实际应用)都可以追究到这种方式。有时，根据测试集的精度衡量，模型表现得非常出色。但是当数据分布突然改变时，模型在部署中会出现灾难性的失败。更隐蔽的是，有时模型的部署本身就是扰乱数据分布的催化剂。假设我们训练了一个贷款申请人违约风险模型，用来预测谁将偿还贷款或违约。这个模型发现申请人的鞋子与违约风险相关(穿牛津鞋的会偿还，穿运动鞋会违约)。此后，这个模型可能倾向于所有穿着牛津鞋的申请人发放贷款，并拒绝所有穿着运动鞋的申请人。

这种情况可能会带来灾难性的后果。首先，一旦模型开始根据鞋类做出决定，顾客就会理解并改变他们的行为。不久，所有的申请人都会穿着牛津鞋，而信用度却没有相应地提高。总而言之，机器学习的许多应用都存在类似的问题：通过基于模型的决策引入环境，我们可能会破坏模型


### 4.9.1 分布偏移的类型
首先，我们考虑数据分布可能发生变化的各种方式，以及为挽救模型性能可能采取的措施。在一个经典的情景中，假设训练数据是从某个分布$pS(x, y)$中采样的，但是测试数据将包含从不同分布$pT(x,y)$中抽取的未标记样本。一个清醒的现实是：如果没有任何关于$pS$和$pT$之间相互关系的假设，学习到一个分类器是不可能的。

考虑一个二元分类问题：区分狗和猫。如果分布可以以任意方式偏移，那么我们的情景允许病态的情况，即输入的分布保持不变：$pS(x)=pT(x)$，但标签全部翻转：$pS(y\mid x)=1 - pT(y \mid x)$。换言之，如果将来所有的“猫”现在都是狗，而我们以前所说的“狗”现在是猫。而此时输入$p(x)$的分布没有任何改变，那么我们就不可能将这种情景与分布完全没有变化的情景区分开来。

**协变量偏移**

在不同分布偏移中，协变量偏移可能是最为广泛研究的。这里我们假设：虽然输入的分布可能随时间而改变，但标签函数(即条件分布$P(y\mid x)$)没有改变。统计学家称之为协变量偏移(covariate shift)，因为这个问题是由于协变量(特征)分布的变化而产生的。虽然有时我们可以在不引用因果关系的情况下对分布偏移进行推断，但在我们认为$x$导致$y$的情况下，协变量偏移是一种自然假设。

考虑一下区分猫和狗的问题：训练数据如下
<div align=center>
<img src='../../pics/4_9_1.jpeg' width='50%'>
</div>

在测试时，我们被要求对下图的图像进行分类
<div align=center>
<img src='../../pics/4_9_2.jpeg' width='50%'>
</div>

训练集由真实照片组成，而测试集只包含卡通图片。假设在一个与测试集的特征有着本质不同的数据集进行训练，如果没有方法来适应新的领域，可能会有麻烦


**标签偏移**

标签偏移(label shift)描述了与协变量偏移相反的问题。这⾥我们假设标签边缘概率$P(y)$可以改变，但是类别条件分布$P(x\mid y)$在不同的领域之间保持不变。当我们认为$y$导致$x$时，标签偏移是⼀个合理的假设。例如，预测患者的疾病，我们可能根据症状来判断，即使疾病的相对流⾏率随着时间的推移⽽变化。标签偏移在这⾥是恰当的假设，因为疾病会引起症状。在另⼀些情况下，标签偏移和协变量偏移假设可以同时成⽴。例如，当标签是确定的，即使y导致x，协变量偏移假设也会得到满⾜。有趣的是，在这些情况下，使⽤基于标签偏移假设的⽅法通常是有利的。这是因为这些⽅法倾向于包含看起来像标签（通常是低维）的对象，⽽不是像输⼊（通常是⾼维的）对象。

**概念偏移**

我们也可能会遇到概念偏移(concept shift):当标签的定义发⽣变化时，就会出现这种问题。这听起来很奇怪——⼀只猫就是⼀只猫，不是吗？然⽽，其他类别会随着不同时间的⽤法⽽发⽣变化。精神疾病的诊断标准、所谓的时髦、以及⼯作头衔等等，都是概念偏移的⽇常映射。事实证明，假如我们环游美国，根据所在的地理位置改变我们的数据来源，我们会发现关于“软饮”名称的分布发⽣了相当⼤的概念偏移，如 图4.9.3所⽰。

<div align=center>
<img src='../../pics/4_9_3.jpeg' width='50%'>
</div>

如果我们要建⽴⼀个机器翻译系统，$P(y\mid x)$的分布可能会因我们的位置不同⽽得到不同的翻译。这个问题可能很难被发现。所以，我们最好可以利⽤在时间或空间上逐渐发⽣偏移的知识。

### 4.9.2 分布偏移示例
在深⼊研究形式体系和算法之前，我们可以讨论⼀些协变量偏移或概念偏移可能并不明显的具体情况。
**医学诊断** 

假设你想设计⼀个检测癌症的算法。你从健康⼈和病⼈那⾥收集数据，然后训练你的算法。它⼯作得很好，有很⾼的精度，然后你得出了你已经准备好在医疗诊断上取得成功的结论。请先别着急。

收集训练数据的分布和在实际中遇到的数据分布可能有很⼤的不同。这件事在⼀个不幸的初创公司⾝上发⽣过，我们中的⼀些作者⼏年前和他们合作过。他们正在研究⼀种⾎液检测⽅法，主要针对⼀种影响⽼年男性的疾病，并希望利⽤他们从病⼈⾝上采集的⾎液样本进⾏研究。然⽽，从健康男性⾝上获取⾎样⽐从系统中已有的病⼈⾝上获取要困难得多。作为补偿，这家初创公司向⼀所⼤学校园内的学⽣征集献⾎，作为开发测试的健康对照样本。然后这家初创公司问我们是否可以帮助他们建⽴⼀个⽤于检测疾病的分类器。

正如我们向他们解释的那样，⽤近乎完美的精度来区分健康和患病⼈群确实很容易。然⽽，这是可能因为受试者在年龄、激素⽔平、体⼒活动、饮⻝、饮酒以及其他许多与疾病⽆关的因素上存在差异。这对检测疾病的分类器可能并不适⽤。这些抽样可能会遇到极端的协变量偏移。此外，这种情况不太可能通过常规⽅法加以纠正。简⾔之，他们浪费了⼀⼤笔钱。

**自动驾驶汽车**

对于⼀家想利⽤机器学习来开发⾃动驾驶汽⻋的公司，⼀个关键部件是“路沿检测器”。由于真实的注释数据获取成本很⾼，他们想出了⼀个“聪明”的想法：将游戏渲染引擎中的合成数据⽤作额外的训练数据。这对从渲染引擎中抽取的“测试数据”⾮常有效，但应⽤在⼀辆真正的汽⻋⾥真是⼀场灾难。正如事实证明的那样，路沿被渲染成⼀种⾮常简单的纹理。更重要的是，所有的路沿都被渲染成了相同的纹理，路沿检测器很快就学习到了这个“特征”。

当美军第⼀次试图在森林中探测坦克时，也发⽣了类似的事情。他们在没有坦克的情况下拍摄了森林的航拍照⽚，然后把坦克开进森林，拍摄了另⼀组照⽚。使⽤这两组数据训练的分类器似乎⼯作得很好。不幸的是，分类器仅仅学会了如何区分有阴影的树和没有阴影的树：第⼀组照⽚是在清晨拍摄的，⽽第⼆组是在中午拍摄的。

**非平稳分布**

当分布变化缓慢并且模型没有得到充分更新时，就会出现更微妙的情况：非平稳分布(nonstationary distribution)。以下是一些典型例子：
- 训练一个计算广告模型，但却没有经常更新
- 建立一个垃圾邮件过滤器，它能很好地检测到所有的垃圾邮件。但是，垃圾邮件发送者变得聪明起来，制造出新的信息，看起来不像我们以前见过的任何垃圾邮件
- 建立一个产品推荐系统，它在整个冬天都有效，但圣诞节过后很久还会继续推荐圣诞帽

### 4.9.3 分布偏移纠正
正如我们所讨论的，在许多情况下训练和测试分布$P(x, y)$是不同的。在一些情况下，不管协变量、标签或概念如何发生偏移，模型都能正常工作。在另一些情况下，我们可以通过运用策略来应对这种偏移，从而做得更好。

**经验风险与实际风险**

首先我们反思一下在模型训练期间到底发生了什么？训练数据$\left\{(x_1, y_1), ..., (x_n, y_n)\right\}$的特征和相关的标签经过迭代，在每个小批量之后更新模型$f$的参数。为了简单起见，我们不考虑正则化，因此极大地降低了训练损失：
$$
\mathop{minimize}\limits_{f} \frac{1}{n} \sum_{i=1}^{n} l(f(x_i), y_i)
$$

其中$l$是损失函数，用来度量：给定标签$y_i$，预测$f(x_i)$的“糟糕程度”。统计学家称上式中的这一项为经验风险。经验风险(empirical risk)是为了近似真实风险(true risk)，整个训练数据上的平均损失，即从其真实分布$p(x, y)$中抽取的所有数据的总体损失的期望值：
$$
E_p(x,y) [l(f(x), y)] = \int \int l(f(x), y) p(x, y) dxdy
$$
然而在实践中，我们通常无法获取总体数据。因此，经验风险最小化是一种使用的机器学习策略，希望能近似最小化真实风险

**协变量偏移纠正**

假设对于带标签的数据$(x_i, y_i)$，我们要评估$P(y \mid x)$。然而观测值$x_i$是从某些源分布$q(x)$中得出的。幸运的是，依赖性假设意味着条件分布保持不变，即$p(y\mid x)=q(y\mid x)$。如果源分布$q(x)$是“错误的”，我们可以通过在真实风险的计算中，使用以下简单的恒等式来进行纠正：
$$
\int \int l(f(x), y) p(y \mid x) p(x) dxdy = \int \int l(f(x), y) q(y\mid x)q(x) \frac{p(x)}{q(x)}dxdy
$$
换句话说，我们需要根据数据来自正确分布与来自错误分布的概率之比，来重新衡量每个数据样本的权重：
$$
\beta_i \mathop{=}\limits^{def} \frac{p(x_i)}{q(x_i)}
$$

将权重$\beta_i$代入每个数据样本$(x_i, y_i)$中，我们可以使用“加权经验风险最小化”来训练模型：
$$
\mathop{minimize}_{f} \frac{1}{n} \sum_{i=1}^n \beta_i l(f(x_i), y_i)
$$
由于不知道这个比率，我们需要估计它。有许多方法都可以用，包括一些花哨的算子理论方法，试图直接使最小化范数或最大熵原理重新校准期望算子。对于任意一种这样的方法，我们都需要从两个分布中抽取样本：“真实”的分布$p$，通过访问测试数据获取：训练集$q$，通过人工合成的很容易获得。请注意，我们只需要特征$x \sim p(x)$，不需要访问标签$y \sim p(y)$。

在这种情况下，有一种非常有效的方法可以得到几乎与原始方法一样好的结果：对数几率回归(logistic regression)。这是用于二元分类的softmax回归的一个特例。综上所述，我们学习了一个分类器来区分从$p(x)$抽取的数据和从$q(x)$抽取的数据。如果无法区分这两个分布，则意味着相关的样本可能来自这两个分布中的任何一个。另一个方面，任何可以很好区分的样本都应该相应地显著增加或减少权重

为了简单起见，假设我们分别从$P(x)$和$q(x)$两个分布中抽取相同数量的样本。现在用$z$标签：从$p$抽取的数据为1，从$q$抽取的数据为-1。然后，混合数据集中的概率由下式给出
$$
P(z=1 \mid x) = \frac{p(x)}{p(x)+ q(x)} and hence \frac{P(z=1 \mid x)}{P(z=-1 \mid x)} = \frac{p(x)}{q(x)}
$$
因此，如果我们使用对数几率回归方法，其中$P(z=1\mid x)=\frac{1}{1 +exp(-h(x))}$(h是一个参数化函数)，则很自然有：
$$
\beta_i = \frac{1/(1+exp(-h(x_i)))}{exp(-h(x_i))/(1+exp(-h(x_i)))} = exp(h(h(x_i)))
$$

因此，我们需要解决两个问题：第一个问题是关于区分来自两个分布的数据集；第二个问题是关于加权经验风险的最小化问题。在这个问题中，我们将对其中的项加权$\beta_i$

现在，我们来看一下完整的协变量偏移纠正算法。假设我们有一个训练集$\left\{ (x_1, y_1), ..., (x_n ,y_n)\right\}$和一个未标记的测试集$\left\{ u_1, ..., u_m\right\}$。对于协变量偏移，我们假设$1 ≤ i ≤ n$的$x_i$来自某个源分布，$u_i$来自目标分布。以下是纠正协变量偏移的典型算法：
- 1.生成一个二元分类训练集：$\left\{(x_1, -1), ..., (x_n, -1), (u_1, 1), ..., (u_m, 1)\right\}$
- 2.用对数几率回归训练二元分类器得到函数$h$
- 3.使用$\beta_i = exp(h(x_i))$或更好的$\beta_i = min(exp(h(x_i)), c)$(c为常量)对训练数据进行加权
- 4.使用权重$\beta_i$进行训练

请注意，上述算法依赖于一个重要的假设：需要目标分布(例如，测试分布)中的每个数据样本在训练时出现的概率非零。如果我们找到$p(x)>0$但$q(x)=0$的点，那么相应的重要性权重会是无穷大

**标签偏移纠正**

假设我们处理的是$k$个类别的分类任务。假设标签的分布随时间变化：$q(x)≠ p(y)$，但类别条件分布保持不变：$q(x \mid y) = p(x\mid y)$。如果源分布$q(y)$是“错误的”，我们可以根据定义的真实风险中的恒等式进行更正：
$$
\int \int l(f(x), y) p(x\mid y) p(y) dxdy = \int \int l(f(x), y) q(x \mid y) q(y) \frac{p(y)}{q(y)} dxdy
$$

这里，重要性权重将对应于标签似然比率
$$
\beta_i \mathop{=}\limits^{def} \frac{p(y_i)}{q(y_i)}
$$

标签偏移的一个好处是，如果我们在源分布上有一个相同好的模型，那么我们可以得到对这些权重的一致估计，而不需要处理周边的其他维度。在深度学习中，输入往往是高维对象(如图像)，而标签通常是低维(如类别)。

现在，我们不能直接计算⽬标数据上的混淆矩阵，因为我们⽆法看到真实环境下的样本的标签，除⾮我们再搭建⼀个复杂的实时标注流程。然⽽，我们所能做的是将所有模型在测试时的预测取平均数，得到平均模型输出$\mu (\hat y) \in \mathbb R^k$，其中第i个元素$\mu (\hat y_i)$是我们模型预测测试集中i的总预测分数。结果表明，如果我们的分类器⼀开始就相当准确，并且⽬标数据只包含我们以前⻅过的类别，以及如果标签偏移假设成⽴（这⾥最强的假设），我们就可以通过求解⼀个简单的线性系统来估计测试集的标签分布
$$
C p(y) = \mu (\hat y)
$$

因为作为一个估计，$\sum_{j=1}^k c_{ij}p(y_j) = \mu (\hat y_i)$对所有$1≤i≤k$成立，其中$P(y_i)$是$k$维标签分布向量$P(y)$的第$y^{th}$元素。如果我们的分类器一开始就足够精确，那么混沌矩阵$C$将是可逆的，进而我们可以得到一个解$p(y)=C^{-1} \mu (\hat y)$

因为我们观测源数据上的标签，所以很容易估计分布$q(y)$。那么对于标签为$y_i$的任何训练样本$i$，我们可以使用我们估计的$p(y_i)/q(y_i)$比率来计算权重$\beta_i$，并将其带入加权经验风险最小化中


**概念偏移纠正**
概念偏移很难⽤原则性的⽅式解决。例如，在⼀个问题突然从“区分猫和狗”偏移为“区分⽩⾊和⿊⾊动物”的情况下，除了从零开始收集新标签和训练，别⽆妙⽅。幸运的是，在实践中这种极端的偏移是罕⻅的。相反，通常情况下，概念的变化总是缓慢的。⽐如下⾯是⼀些例⼦：
- 在计算⼴告中，新产品推出后，旧产品变得不那么受欢迎了。这意味着⼴告的分布和受欢迎程度是逐渐
变化的，任何点击率预测器都需要随之逐渐变化。
- 由于环境的磨损，交通摄像头的镜头会逐渐退化，影响摄像头的图像质量。
- 新闻内容逐渐变化（即新新闻的出现）。

在这种情况下，我们可以使⽤与训练⽹络相同的⽅法，使其适应数据的变化。换⾔之，我们使⽤新数据更新
现有的⽹络权重，⽽不是从头开始训练

### 4.9.4 学习问题的分类法
有了如何处理分布变化的知识，我们现在可以考虑机器学习问题形式化的其他⽅⾯。

**批量学习**

在批量学习(batch learning)中，我们可以访问⼀组训练特征和标签$\left\{(x_1, y_1), . . . ,(x_n, y_n)\right\}$，我们使⽤这些特性和标签训练$f(x)$。然后，我们部署此模型来对来⾃同⼀分布的新数据$(x, y)$进⾏评分。例如，我们可以根据猫和狗的⼤量图⽚训练猫检测器。⼀旦我们训练了它，我们就把它作为智能猫⻔计算视觉系统的⼀部分，来控制只允许猫进⼊。然后这个系统会被安装在客⼾家中，基本再也不会更新。

**在线学习**

除了“批量”地学习，我们还可以单个“在线”学习数据$(x_i, y_i)$。更具体地说，我们⾸先观测到$x_i$，然后我们得出⼀个估计值$f(x_i)$，只有当我们做到这⼀点后，我们才观测到$y_i$。然后根据我们的决定，我们会得到奖励或损失。许多实际问题都属于这⼀类。例如，我们需要预测明天的股票价格，这样我们就可以根据这个预测进⾏交易。在⼀天结束时，我们会评估我们的预测是否盈利。换句话说，在在线学习(online learning)中，我们有以下的循环。在这个循环中，给定新的观测结果，我们会不断地改进我们的模型。
$$
model f_t \rightarrow data x_t \rightarrow estimate f_t(x_t) \rightarrow observation y_t \rightarrow loss l(y_t, f_t(x_t)) \rightarrow model f_{t+1}
$$

**⽼虎机**

⽼⻁机(bandits)是上述问题的⼀个特例。虽然在⼤多数学习问题中，我们有⼀个连续参数化的函数$f$(例如，⼀个深度⽹络)。但在⼀个⽼⻁机问题中，我们只有有限数量的⼿臂可以拉动。也就是说，我们可以采取的⾏动是有限的。对于这个更简单的问题，可以获得更强的最优性理论保证，这并不令⼈惊讶。我们之所以
列出它，主要是因为这个问题经常被视为⼀个单独的学习问题的情景。

**控制**

在很多情况下，环境会记住我们所做的事。不⼀定是以⼀种对抗的⽅式，但它会记住，⽽且它的反应将取决于之前发⽣的事情。例如，咖啡锅炉控制器将根据之前是否加热锅炉来观测到不同的温度。在这种情况下，PID(⽐例—积分—微分)控制器算法是⼀个流⾏的选择。同样，⼀个⽤⼾在新闻⽹站上的⾏为将取决于之前向她展⽰的内容(例如，⼤多数新闻她只阅读⼀次)。许多这样的算法形成了⼀个环境模型，在这个模型中，他们的⾏为使得他们的决策看起来不那么随机。近年来，控制理论(如PID的变体)也被⽤于⾃动调整超参数，以获得更好的解构和重建质量，提⾼⽣成⽂本的多样性和⽣成图像的重建质量。

**强化学习**
强化学习(reinforcement learning)强调如何基于环境⽽⾏动，以取得最⼤化的预期利益。国际象棋、围棋、西洋双陆棋或星际争霸都是强化学习的应⽤实例。再⽐如，为⾃动驾驶汽⻋制造⼀个控制器，或者以其他⽅式对⾃动驾驶汽⻋的驾驶⽅式做出反应(例如，试图避开某物体，试图造成事故，或者试图与其合作)。

**考虑到环境**

上述不同情况之间的⼀个关键区别是：在静⽌环境中可能⼀直有效的相同策略，在环境能够改变的情况下可能不会始终有效。例如，⼀个交易者发现的套利机会很可能在他开始利⽤它时就消失了。环境变化的速度和⽅式在很⼤程度上决定了我们可以采⽤的算法类型。例如，如果我们知道事情只会缓慢地变化，就可以迫使任何估计也只能缓慢地发⽣改变。如果我们知道环境可能会瞬间发⽣变化，但这种变化⾮常罕⻅，我们就可以在使⽤算法时考虑到这⼀点。当⼀个数据科学家试图解决的问题会随着时间的推移⽽发⽣变化时，这些类型的知识⾄关重要。

### 4.9.5 机器学习中的公平、责任和透明度
最后，重要的是，当你部署机器学习系统时，你不仅仅是在优化⼀个预测模型，⽽你通常是在提供⼀个会被⽤来（部分或完全）进⾏⾃动化决策的⼯具。这些技术系统可能会通过其进⾏的决定⽽影响到每个⼈的⽣活。从考虑预测到决策的⻜跃不仅提出了新的技术问题，⽽且还提出了⼀系列必须仔细考虑的伦理问题。如果我们正在部署⼀个医疗诊断系统，我们需要知道它可能适⽤于哪些⼈群，哪些⼈群可能⽆效。忽视对⼀个亚群体的幸福的可预⻅⻛险可能会导致我们执⾏劣质的护理⽔平。此外，⼀旦我们规划整个决策系统，我们必须退后⼀步，重新考虑如何评估我们的技术。在这个视野变化所导致的结果中，我们会发现精度很少成为合适的衡量标准。例如，当我们将预测转化为⾏动时，我们通常会考虑到各种⽅式犯错的潜在成本敏感性。举个例⼦：将图像错误地分到某⼀类别可能被视为种族歧视，⽽错误地分到另⼀个类别是⽆害的，那么我们可能需要相应地调整我们的阈值，在设计决策⽅式时考虑到这些社会价值。我们还需要注意预测系统如何导致反馈循环。例如，考虑预测性警务系统，它将巡逻⼈员分配到预测犯罪率较⾼的地区。很容易看出⼀种令⼈担忧的模式是如何出现的：
1. 犯罪率⾼的社区会得到更多的巡逻。
2. 因此，在这些社区中会发现更多的犯罪⾏为，输⼊可⽤于未来迭代的训练数据。
3. ⾯对更多的积极因素，该模型预测这些社区还会有更多的犯罪。
4. 下⼀次迭代中，更新后的模型会更加倾向于针对同⼀个地区，这会导致更多的犯罪⾏为被发现等等。

### 4.9.6 ⼩结
- 在许多情况下，训练集和测试集并不来⾃同⼀个分布。这就是所谓的分布偏移。
- 真实⻛险是从真实分布中抽取的所有数据的总体损失的预期。然⽽，这个数据总体通常是⽆法获得的。经验⻛险是训练数据的平均损失，⽤于近似真实⻛险。在实践中，我们进⾏经验⻛险最⼩化。
- 在相应的假设条件下，可以在测试时检测并纠正协变量偏移和标签偏移。在测试时，不考虑这种偏移可能会成为问题。
- 在某些情况下，环境可能会记住⾃动操作并以令⼈惊讶的⽅式做出响应。在构建模型时，我们必须考虑到这种可能性，并继续监控实时系统，并对我们的模型和环境以意想不到的⽅式纠缠在⼀起的可能性持开放态度。